In [1]:
import pandas as pd
import numpy as np
import requests
import time

In [2]:
API_KEY = <YOUR-API-KEY> # go to their website and quickly get one

# data folder
# note: make sure this folder exists
DATA_FOLDER ="./data" 

SYMBOL = 'AAPL'
RESOLUTION = "30min" # '1min', '5min', '15min', '30min', '60min'

START_YEAR = 2009
END_YEAR = 2010


In [3]:
## full intraday
download_bars = pd.DataFrame()

years = np.arange(START_YEAR, END_YEAR)
print(years)

for year in years:
    for month in range (1, 13):
        print (f'year: {year}, month: {month:02d}')

        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={SYMBOL}&interval={RESOLUTION}&month={year}-{month:02d}&outputsize=full&apikey={API_KEY}' 
        
        download = requests.get(url)
        
        if "Thank you for using" in  download.text:
            print("rate limiting hit")
            time.sleep(61)
            download = requests.get(url)
        
        data = download.json()
        
        bars = pd.DataFrame(data[f"Time Series ({RESOLUTION})"]).T
        
        download_bars = pd.concat([download_bars, bars])


[2009]
year: 2009, month: 01
year: 2009, month: 02
year: 2009, month: 03
year: 2009, month: 04
year: 2009, month: 05
year: 2009, month: 06
rate limiting hit
year: 2009, month: 07
year: 2009, month: 08
year: 2009, month: 09
year: 2009, month: 10
year: 2009, month: 11
rate limiting hit
year: 2009, month: 12


In [4]:
download_bars

,1. open,2. high,3. low,4. close,5. volume
2009-01-30 19:30:00,2.7380,2.7420,2.7300,2.7380,133336
2009-01-30 19:00:00,2.7340,2.7410,2.7280,2.7370,61796
2009-01-30 18:30:00,2.7350,2.7390,2.7280,2.7320,61208
2009-01-30 18:00:00,2.7320,2.7410,2.7260,2.7340,124348
2009-01-30 17:30:00,2.7340,2.7370,2.7260,2.7310,91084
...,...,...,...,...,...
2009-12-01 07:00:00,6.1150,6.1230,6.0990,6.1090,96656
2009-12-01 06:00:00,6.1160,6.1220,6.1040,6.1140,12600
2009-12-01 05:30:00,6.1160,6.1220,6.1040,6.1140,14000
2009-12-01 04:30:00,6.1100,6.1160,6.0980,6.1080,2800


In [5]:

# column and index renaming
download_bars.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
download_bars.index.name = 'Timestamp'
download_bars

,Open,High,Low,Close,Volume
Timestamp,,,,,
2009-01-30 19:30:00,2.7380,2.7420,2.7300,2.7380,133336
2009-01-30 19:00:00,2.7340,2.7410,2.7280,2.7370,61796
2009-01-30 18:30:00,2.7350,2.7390,2.7280,2.7320,61208
2009-01-30 18:00:00,2.7320,2.7410,2.7260,2.7340,124348
2009-01-30 17:30:00,2.7340,2.7370,2.7260,2.7310,91084
...,...,...,...,...,...
2009-12-01 07:00:00,6.1150,6.1230,6.0990,6.1090,96656
2009-12-01 06:00:00,6.1160,6.1220,6.1040,6.1140,12600
2009-12-01 05:30:00,6.1160,6.1220,6.1040,6.1140,14000


In [6]:
# type conversion and column selection
all_bars = pd.DataFrame()
ohlc_columns = ['Open', 'High', 'Low', 'Close']
all_bars[ohlc_columns] = download_bars[ohlc_columns].apply(pd.to_numeric)
all_bars.index = pd.to_datetime(all_bars.index)
all_bars = all_bars.sort_index()
all_bars.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6938 entries, 2009-01-02 07:30:00 to 2009-12-31 19:30:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    6938 non-null   float64
 1   High    6938 non-null   float64
 2   Low     6938 non-null   float64
 3   Close   6938 non-null   float64
dtypes: float64(4)
memory usage: 271.0 KB


In [7]:
# saving to csv
all_bars.to_csv(f"{DATA_FOLDER}/alphavantage_{SYMBOL}.csv")